In [1]:
import struct
from pathlib import Path

PATH = Path.cwd()


def read_file_header(file_path, n=256):
    with open(file_path, "rb") as f:
        return f.read(n)


def parse_tiff_header(header_bytes):
    byte_order = header_bytes[0:2]
    if byte_order == b"II":
        byte_order = "Little Endian"
    elif byte_order == b"MM":
        byte_order = "Big Endian"
    else:
        byte_order = "Unknown"
    version = struct.unpack_from("<H", header_bytes, 2)[0]
    first_ifd_offset = struct.unpack_from("<I", header_bytes, 4)[0]
    return {
        "Byte Order": byte_order,
        "Version": version,
        "First IFD Offset": first_ifd_offset,
    }


header_lenna01 = read_file_header(PATH / "Lenna01.tiff")
header_lenna02 = read_file_header(PATH / "Lenna02.tiff")
parsed_header_lenna01 = parse_tiff_header(header_lenna01)
parsed_header_lenna02 = parse_tiff_header(header_lenna02)

parsed_header_lenna01, parsed_header_lenna02

({'Byte Order': 'Little Endian', 'Version': 42, 'First IFD Offset': 8},
 {'Byte Order': 'Little Endian', 'Version': 42, 'First IFD Offset': 8})

In [2]:
def parse_ifd_entry(entry_bytes, byte_order_format):
    tag, typ, count, value_offset = struct.unpack(
        byte_order_format + "HHII", entry_bytes
    )
    return {"Tag": tag, "Type": typ, "Count": count, "Value_Offset": value_offset}


def parse_ifd(header_bytes, offset, byte_order_format="<"):
    num_entries = struct.unpack_from(byte_order_format + "H", header_bytes, offset)[0]
    offset += 2
    entries = []
    for _ in range(num_entries):
        entry_bytes = header_bytes[offset : offset + 12]
        entries.append(parse_ifd_entry(entry_bytes, byte_order_format))
        offset += 12
    next_ifd_offset = struct.unpack_from(byte_order_format + "I", header_bytes, offset)[
        0
    ]
    return entries, next_ifd_offset


byte_order_format = "<"
parsed_ifd_lenna01, next_ifd_offset_lenna01 = parse_ifd(
    header_lenna01, parsed_header_lenna01["First IFD Offset"], byte_order_format
)
parsed_ifd_lenna02, next_ifd_offset_lenna02 = parse_ifd(
    header_lenna02, parsed_header_lenna02["First IFD Offset"], byte_order_format
)

parsed_ifd_lenna01, next_ifd_offset_lenna01, parsed_ifd_lenna02, next_ifd_offset_lenna02

([{'Tag': 254, 'Type': 4, 'Count': 1, 'Value_Offset': 0},
  {'Tag': 256, 'Type': 3, 'Count': 1, 'Value_Offset': 512},
  {'Tag': 257, 'Type': 3, 'Count': 1, 'Value_Offset': 512},
  {'Tag': 258, 'Type': 3, 'Count': 1, 'Value_Offset': 8},
  {'Tag': 259, 'Type': 3, 'Count': 1, 'Value_Offset': 1},
  {'Tag': 262, 'Type': 3, 'Count': 1, 'Value_Offset': 1},
  {'Tag': 273, 'Type': 4, 'Count': 1, 'Value_Offset': 25202},
  {'Tag': 274, 'Type': 3, 'Count': 1, 'Value_Offset': 1},
  {'Tag': 277, 'Type': 3, 'Count': 1, 'Value_Offset': 1},
  {'Tag': 278, 'Type': 3, 'Count': 1, 'Value_Offset': 512},
  {'Tag': 279, 'Type': 4, 'Count': 1, 'Value_Offset': 262144},
  {'Tag': 282, 'Type': 5, 'Count': 1, 'Value_Offset': 254},
  {'Tag': 283, 'Type': 5, 'Count': 1, 'Value_Offset': 262},
  {'Tag': 296, 'Type': 3, 'Count': 1, 'Value_Offset': 2},
  {'Tag': 305, 'Type': 2, 'Count': 31, 'Value_Offset': 270},
  {'Tag': 306, 'Type': 2, 'Count': 20, 'Value_Offset': 302},
  {'Tag': 700, 'Type': 1, 'Count': 15083, 'Valu